In [ ]:
import pyart
import fsspec
from metpy.plots import USCOUNTIES
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
import numpy as np
warnings.filterwarnings("ignore")
fs = fsspec.filesystem("s3", anon=True) 
import cartopy.io.shapereader as shpreader
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import math
import matplotlib.ticker as mticker

In [ ]:
## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

In [ ]:
plt.rcParams['axes.facecolor'] = '#0e1111'
plt.rcParams['figure.facecolor'] = '#232b2b'
plt.rcParams['text.color'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams["figure.figsize"] = (6.5, 6.5) #Utilize plt.rcParams to configure the way your plot looks.

In [ ]:
fs = fsspec.filesystem("s3", anon=True) #Set up AWS S3 file system

In [ ]:
#s3://noaa-nexrad-level2/year/month/date/radarsite/{radarsite}{year}{month}{date}_{hour}{minute}{second}_V06 #Pull NEXRAD Level 2 data from AWS S3 for our plot

In [ ]:
files = sorted(fs.glob("s3://noaa-nexrad-level2/2013/05/20/KTLX/KTLX20130520_201643_V06.gz*"))
files #specify the file you want to plot

In [ ]:
reader1 = shpreader.Reader("/mnt/path/to/shapefile/")
reader2 = shpreader.Reader("/mnt/path/to/shapefile/") 
#Set up some readers for the shapefiles we will be plotting with CartoPy
#Extract the shapefiles.zip and put them in a safe directory and replace "/mnt/path/to/shapefile" with your specific paths

In [ ]:
interstates = list(reader1.geometries())
INTERSTATES = cfeature.ShapelyFeature(interstates, ccrs.PlateCarree())

counties = list(reader2.geometries())
COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

#Define our readers and shapefiles so we can use them in the plot later

In [ ]:
radar = pyart.io.read_nexrad_archive("s3://noaa-nexrad-level2/2013/05/20/KTLX/KTLX20130520_201643_V06.gz*")
list(radar.fields)

#Create a PyART module to read the data with pyart.io.read_nexrad_archive and list the fields

In [ ]:
['velocity',
 'spectrum_width',
 'differential_reflectivity',
 'differential_phase',
 'reflectivity',
 'cross_correlation_ratio']

In [ ]:
fig = plt.figure()
#specify the figure

In [ ]:
radar_lat = radar.latitude['data'] = np.array([35.333])
radar_lon = radar.longitude['data'] = np.array([-97.278])
radar_name = radar.metadata['instrument_name']
#Grab the radar lat/lon and radar metadata


In [ ]:
gatefilter = pyart.filters.GateFilter(radar)
gatefilter.exclude_transition()
gatefilter.exclude_invalid("velocity")
gatefilter.exclude_outside("reflectivity", 0, 80)
gatefilter.exclude_outside("reflectivity", 0, 80)
gatefilter.exclude_below('reflectivity', 0)

dealias_data = pyart.correct.dealias_region_based(radar, gatefilter=gatefilter)
radar.add_field("corrected_velocity", dealias_data)

#Apply a gatefilter to make our data look nice and also apply a region dealias so the velocity data plotted is correct.

In [ ]:
sweep = 0
#Specify a sweep in the file

In [ ]:
index_at_start = radar.sweep_start_ray_index['data'][sweep]
time_at_start_of_radar = pyart.io.cfradial.netCDF4.num2date(radar.time['data'][index_at_start], 
radar.time['units'])
formatted_date = time_at_start_of_radar.strftime('%m-%d-%Y %H:%MZ')
#Make sure we grab the UTC time

In [ ]:
slice_indices = radar.get_slice(sweep)
max_ref = radar.fields['reflectivity']['data'][slice_indices].max()
elev_angle = (round(radar.elevation['data'][slice_indices].mean(), 2)) 
deg_sign = u'\N{DEGREE SIGN}'
#Grab the sweep elevation angle, and max values in the 'reflectivity' field for our plot title.

In [ ]:
ax1 = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax1.add_feature(INTERSTATES, facecolor='none', edgecolor='yellow')
ax1.add_feature(COUNTIES, facecolor='none', edgecolor='red')
display = pyart.graph.RadarMapDisplay(radar)
ref_map = display.plot_ppi_map('reflectivity',
                               sweep=0,
                               vmin=-10,
                               vmax=70, 
                               ax=ax1,
                               cmap='pyart_NWSRef',
                               colorbar_label='', title='', colorbar_flag=False)
#lets begin plotting! Using cartopy in matplotlib ensures we have a clean projection to map our NEXRAD data to.
#Using the shapefiles imported earlier, we can plot high resolution counties and interstates as well!

In [ ]:
display.plot_range_ring(50., linestyle='dashed', color='gainsboro', lw=1)
display.plot_range_ring(150., linestyle='solid', color='gainsboro', lw=1)
display.plot_range_ring(250., linestyle='dashdot', color='gainsboro', lw=1)
display.plot_range_ring(350., linestyle='dashed', color='gainsboro', lw=1)

#Plot range rings

In [ ]:
ax1.set_title('Level-II\nRadar: ' + radar_name, fontsize=11, loc='left')
ax1.set_title(str(formatted_date), fontsize=11, loc='center', fontweight='bold')
ax1.set_title('$Z_{e}$ | Max: ' + str(max_ref) + ' dBz ' + '\nElev Angle: ' + str(elev_angle) + deg_sign, fontsize=11, loc='right')

#Set a pretty plot title up

cax = fig.add_axes([ax1.get_position().x0, ax1.get_position().y0 - 0.09, ax1.get_position().width, 0.05])
display.plot_colorbar(mappable=None, field=None, label='($Z_{e}$)', orient='horizontal', cax=cax, ax=ax1, fig=None, ticks=[-10, 0, 10, 20, 30, 40, 50, 60, 70], ticklabs=None, extend='both')
cax.xaxis.label.set_color('white')

#Plot a nice colorbar that rests directly under your plot with custom ticklabels


In [ ]:
ax1.add_feature(cfeature.LAND.with_scale('10m'))
ax1.add_feature(cfeature.OCEAN.with_scale('10m'))
ax1.add_feature(cfeature.LAKES.with_scale('10m'))

resol = '10m'
land = cfeature.NaturalEarthFeature('physical', 'land', scale=resol, facecolor=cfeature.COLORS['land'])
ax1.add_feature(land, facecolor='#5A4B41', zorder=0)

#Add some land and lakes to your plot with a custom land facecolor

In [ ]:
dtor = math.pi/180.0
max_range = 350
maxrange_meters = max_range * 1000.
meters_to_lat = 1. / 111177.
meters_to_lon =  1. / (111177. * math.cos(radar_lat * dtor))

#Do some fancy math for our azimuth lines

In [ ]:
for azi in range(0, 360, 45):  #45 degree intervals
    azimuth = 90. - azi
    dazimuth = azimuth * dtor
    lon_maxrange = radar_lon + math.cos(dazimuth) * meters_to_lon * maxrange_meters
    lat_maxrange = radar_lat + math.sin(dazimuth) * meters_to_lat * maxrange_meters
    display.plot_line_geo([radar_lon, lon_maxrange], [radar_lat, lat_maxrange], linestyle='-', lw=0.5, color='gainsboro')

max_values = [50, 150, 350]     


#Set up our azimuth lines and plot them

In [ ]:
azim = 90. - 30
dazim = azim * dtor
for i in max_values:
    max_m = i * 1000.
    lon_max = radar_lon + math.cos(dazim) * meters_to_lon * max_m
    lat_max = radar_lat + math.sin(dazim) * meters_to_lat * max_m
    display.plot_point(lon_max, lat_max, symbol='None', label_text=' '+str(i)+'km')

display.plot_point(radar.longitude['data'][0], radar.latitude['data'][0],color='k',label_text='KTLX', symbol='*', markersize=10)

#Display some distance markers and a nice plot point and radar name at the radar site

In [ ]:
output_png = 'PYART.png'
dpi = 800
plt.savefig(output_png, dpi=dpi)
#Create an output image of the plot in whatever directory youre currently in 


plt.show()
#show the plot